# Baseline de Modelagem – Gravidade de Sinistros 2007

Este notebook implementa um **modelo baseline** para prever a gravidade do sinistro.

### Definição de gravidade
Vamos considerar um sinistro como **grave** quando houver **qualquer vítima não ilesa**.

Com base nas colunas disponíveis:
- `mortos`
- `feridos_leves`
- `feridos_graves`
- (não usamos `ilesos` para gravidade)

Definimos a variável binária `grave` assim:
- `grave = 1` se `mortos > 0` **ou** `feridos_leves > 0` **ou** `feridos_graves > 0`
- `grave = 0` caso contrário.

Depois usamos essa coluna `grave` como variável-alvo em um modelo baseline de **Regressão Logística**.

In [ ]:
# Imports básicos
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report,
    confusion_matrix
)

import pyarrow.parquet as pq  

RANDOM_STATE = 42
pd.set_option('display.max_columns', None)

## 1. Carregamento dos dados

Usamos `pyarrow` diretamente para evitar conflitos entre pandas e pyarrow na leitura do arquivo.
Assumimos que o arquivo `sinistros2007.parquet` está na mesma pasta deste notebook.

In [ ]:
DATA_PATH = 'sinistros2007.parquet' 

table = pq.read_table(DATA_PATH)
df = table.to_pandas()
df["faixa_horaria"] = pd.cut(
    df["hora"],
    bins=[0, 5, 11, 17, 23],
    labels=["madrugada", "manha", "tarde", "noite"],
    include_lowest=True
)


if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

print("Formato do dataset:", df.shape)
df.head()

Formato do dataset: (79811, 32)


,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,ano,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,pop_2006,hora,mes,fim_semana,dia,faixa_horaria
0,1051130,2007-02-12,Segunda,02:10:00,MA,135,11.0,SAO LUIS,Animais na Pista,Atropelamento de animal,Com V�timas Fatais,Plena noite,Crescente,Ceu Claro,Simples,Reta,Urbano,2007,5,2,2,1,0,0,3,1,998385.0,2,2,0,12,madrugada
1,1066824,2007-11-20,Ter�a,05:30:00,CE,222,30.8,CAUCAIA,Defeito mec�nico em ve�culo,Capotamento,Com V�timas Feridas,Amanhecer,Decrescente,Ceu Claro,Dupla,Reta,Rural,2007,1,0,1,0,0,0,1,1,313584.0,5,11,1,20,madrugada
2,1333766,2007-07-17,Ter�a,14:00:00,GO,70,372.0,MONTES CLAROS DE GOIAS,Falta de aten��o,Colis�o lateral,Com V�timas Feridas,Pleno dia,Crescente,Ceu Claro,Simples,Reta,Rural,2007,2,0,0,1,1,0,1,2,7652.0,14,7,1,17,tarde
3,173714,2007-01-01,Segunda,00:03:00,RJ,116,305.5,RESENDE,Animais na Pista,Atropelamento de animal,Sem V�timas,Plena noite,Crescente,Chuva,Dupla,Reta,Urbano,2007,1,0,0,0,1,0,0,1,119729.0,0,1,0,1,madrugada
4,173715,2007-01-01,Segunda,00:50:00,RJ,101,321.5,NITEROI,N�o guardar dist�ncia de seguran�a,Colis�o traseira,Sem V�timas,Plena noite,Decrescente,Chuva,Dupla,Reta,Urbano,2007,2,0,0,0,2,0,0,2,476669.0,0,1,0,1,madrugada


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79811 entries, 0 to 79810
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      79811 non-null  int64         
 1   data_inversa            79811 non-null  datetime64[ns]
 2   dia_semana              79811 non-null  object        
 3   horario                 79811 non-null  object        
 4   uf                      79811 non-null  object        
 5   br                      79811 non-null  int64         
 6   km                      79811 non-null  float64       
 7   municipio               79811 non-null  object        
 8   causa_acidente          79811 non-null  object        
 9   tipo_acidente           79811 non-null  object        
 10  classificacao_acidente  79811 non-null  object        
 11  fase_dia                79811 non-null  object        
 12  sentido_via             79811 non-null  object

In [ ]:
df.columns.tolist()

['id',
 'data_inversa',
 'dia_semana',
 'horario',
 'uf',
 'br',
 'km',
 'municipio',
 'causa_acidente',
 'tipo_acidente',
 'classificacao_acidente',
 'fase_dia',
 'sentido_via',
 'condicao_metereologica',
 'tipo_pista',
 'tracado_via',
 'uso_solo',
 'ano',
 'pessoas',
 'mortos',
 'feridos_leves',
 'feridos_graves',
 'ilesos',
 'ignorados',
 'feridos',
 'veiculos',
 'pop_2006',
 'hora',
 'mes',
 'fim_semana',
 'dia',
 'faixa_horaria']

## 2. Criação da variável-alvo `grave`

Conforme combinado:
- Acidente é **grave** se existe pelo menos uma vítima não ilesa.
- Usamos explicitamente as colunas `mortos`, `feridos_leves`, `feridos_graves`.

Se alguma dessas colunas não existir, um erro será lançado para você ajustar.

In [ ]:
required_cols = ["mortos", "feridos_leves", "feridos_graves"]

for c in required_cols:
    if c not in df.columns:
        raise ValueError(f"Coluna obrigatória para gravidade não encontrada: {c!r}")

df["grave"] = (
    (df["mortos"] > 0) |
    (df["feridos_leves"] > 0) |
    (df["feridos_graves"] > 0)
).astype(int)

print("Distribuição de 'grave':")
print(df["grave"].value_counts())
print("\nDistribuição relativa de 'grave':")
print(df["grave"].value_counts(normalize=True))

Distribuição de 'grave':
grave
0    48717
1    31094
Name: count, dtype: int64

Distribuição relativa de 'grave':
grave
0    0.610405
1    0.389595
Name: proportion, dtype: float64


## 3. Separação em features e alvo + split treino/teste

Agora usamos `grave` como nossa variável-alvo para o problema de classificação binária.

In [23]:

TARGET_COL = "grave"

# Colunas que VAZAM a resposta (devem ser removidas do treino)
cols_vazamento = [
    "mortos",
    "feridos",
    "feridos_leves",
    "feridos_graves",
    "ilesos",
    "ignorados",
    "pessoas",
    "classificacao_acidente",
    "tipo_acidente",
    "horario",
    "hora",
    "municipio",
    "data_inversa",
    "uf"
          
]

cols_vazamento += ["id", "pop_2006"]




# Mantém apenas colunas que NÃO vazam o resultado
X = df.drop(columns=[TARGET_COL] + cols_vazamento, errors="ignore")
y = df[TARGET_COL]

print("Colunas removidas:", cols_vazamento)
print("Colunas restantes:", X.columns.tolist())

# Split treino/teste (estratificado)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y
)

X_train.shape, X_test.shape

Colunas removidas: ['mortos', 'feridos', 'feridos_leves', 'feridos_graves', 'ilesos', 'ignorados', 'pessoas', 'classificacao_acidente', 'tipo_acidente', 'horario', 'hora', 'municipio', 'data_inversa', 'uf', 'id', 'pop_2006']
Colunas restantes: ['dia_semana', 'br', 'km', 'causa_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo', 'ano', 'veiculos', 'mes', 'fim_semana', 'dia', 'faixa_horaria']


((63848, 16), (15963, 16))

## 4. Pré-processamento (Feature Engineering básica)

Neste baseline vamos aplicar apenas transformações **básicas**:
- Padronização (`StandardScaler`) para variáveis numéricas.
- One-hot encoding (`OneHotEncoder`) para variáveis categóricas.

A detecção de numéricas vs categóricas é feita automaticamente a partir dos tipos de dados.

In [ ]:
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

print("Numéricas:", numeric_features)
print("Categóricas:", categorical_features)

Numéricas: ['br', 'km', 'ano', 'veiculos']
Categóricas: ['dia_semana', 'causa_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo', 'mes', 'fim_semana', 'dia', 'faixa_horaria']


In [25]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


## 5. Modelo Baseline – Regressão Logística

A Regressão Logística é um modelo **simples e interpretável**, adequado para servir
como baseline na tarefa de classificação da gravidade dos sinistros.

Depois vamos comparar seu desempenho com modelos mais complexos (Random Forest,
Gradient Boosting, Redes Neurais, ensembles etc.).

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg_clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
    max_iter=2000,
    random_state=42,
    C=0.1,              # regularização mais forte
    penalty="l2",
    solver="lbfgs"
)
)
])

# 2) Treinar o modelo com os dados LIMPOS (sem mortos/feridos/classificacao_acidente,etc)
log_reg_clf.fit(X_train, y_train)

# 3) Extrair nomes das features após o OneHot/Scaler
feature_names = log_reg_clf.named_steps["preprocessor"].get_feature_names_out()
coefs = log_reg_clf.named_steps["classifier"].coef_[0]

# 4) Montar tabela ordenada pelas maiores magnitudes
import pandas as pd

coef_df = pd.DataFrame({
    "feature": feature_names,
    "coeficiente": coefs
}).sort_values(by="coeficiente", key=abs, ascending=False)

coef_df.head(15)


,feature,coeficiente
11,cat__causa_acidente_Animais na Pista,-0.988945
18,cat__causa_acidente_N�o guardar dist�ncia de s...,-0.820514
17,cat__causa_acidente_Ingest�o de �lcool,0.635748
14,cat__causa_acidente_Desobedi�ncia � sinaliza��o,0.488684
19,cat__causa_acidente_Ultrapassagem indevida,0.456877
20,cat__causa_acidente_Velocidade incompat�vel,0.403990
12,cat__causa_acidente_Defeito mec�nico em ve�culo,-0.358928
28,cat__condicao_metereologica_Chuva,-0.355546
36,cat__tipo_pista_M�ltipla,-0.304199
37,cat__tipo_pista_Simples,0.247068


In [27]:
from sklearn.metrics import classification_report, confusion_matrix

# Previsões
y_pred = log_reg_clf.predict(X_test)

# Relatório completo
print("=== REGRESSÃO LOGÍSTICA ===")
print(classification_report(y_test, y_pred))

# Matriz de confusão
print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred))


=== REGRESSÃO LOGÍSTICA ===
              precision    recall  f1-score   support

           0       0.66      0.87      0.75      9744
           1       0.58      0.29      0.38      6219

    accuracy                           0.64     15963
   macro avg       0.62      0.58      0.56     15963
weighted avg       0.62      0.64      0.60     15963

Matriz de confusão:
[[8441 1303]
 [4442 1777]]


## Análise de Importância das Variáveis e Respostas às Perguntas de Pesquisa

    A análise de importância das variáveis foi realizada utilizando um modelo de Regressão Logística sem vazamento de dados, excluindo-se previamente todas as variáveis que definem diretamente a gravidade do acidente (número de mortos, feridos, pessoas envolvidas, classificação oficial e tipo de acidente), bem como variáveis de localização excessivamente específicas (município), de modo a garantir interpretabilidade e validade científica dos resultados. Após esse controle, o modelo passou a considerar apenas fatores contextuais, estruturais e comportamentais.
    Os resultados indicam que os principais fatores associados ao aumento da gravidade dos acidentes de trânsito em rodovias federais em 2007 estão relacionados ao comportamento do condutor e à infraestrutura viária. A ingestão de álcool destacou-se como a variável de maior impacto positivo na probabilidade de ocorrência de acidentes graves, evidenciando sua forte associação com fatalidades e lesões severas. Outras causas comportamentais relevantes foram a desobediência à sinalização, ultrapassagens indevidas e velocidade incompatível com a via, todas apresentando coeficientes positivos, o que indica maior risco de gravidade.
    No aspecto estrutural, observou-se que vias de pista simples apresentaram maior propensão à ocorrência de acidentes graves quando comparadas a vias de pista múltipla, resultado coerente com a maior exposição ao risco de colisões frontais e ausência de separação física entre fluxos opostos. Em contrapartida, vias de pista múltipla apresentaram associação negativa com a gravidade dos acidentes, sugerindo maior nível de proteção estrutural.
    Alguns fatores apresentaram associações aparentemente contraintuitivas, como a condição meteorológica de chuva e a presença de animais na pista, ambos associados a menores coeficientes de gravidade. Esse comportamento pode ser explicado pela redução da velocidade média adotada pelos condutores em condições adversas, o que diminui a energia de impacto e, consequentemente, a severidade dos sinistros. De forma semelhante, ocorrências envolvendo defeitos mecânicos também tenderam a apresentar menor gravidade, possivelmente por ocorrerem em velocidades mais baixas.
    No eixo temporal, acidentes registrados durante o pleno dia e no período da manhã exibiram menor probabilidade de gravidade, o que pode ser atribuído à maior visibilidade, ao maior fluxo de veículos e à menor incidência de consumo de álcool nesses períodos. Esses resultados reforçam a importância das variáveis temporais como fatores explicativos relevantes.
    Em resposta à primeira pergunta de pesquisa, conclui-se que a gravidade dos acidentes é fortemente influenciada por fatores comportamentais (álcool, imprudência), estruturais (tipo de pista) e ambientais (condições meteorológicas), e não apenas por localização geográfica. Quanto à segunda pergunta, os resultados demonstram que é possível prever a gravidade do acidente utilizando apenas características contextuais e estruturais da ocorrência, sem a necessidade de informações diretas sobre vítimas. Essa constatação evidencia o potencial de modelos preditivos como ferramentas de apoio à formulação de políticas públicas e estratégias preventivas de segurança viária.
    A seleção de variáveis foi realizada com base em importância estatística e interpretação semântica, garantindo que apenas fatores com significado causal e operacional fossem mantidos no modelo. Tal abordagem permitiu maior transparência na análise e evitou que o classificador aprendesse padrões artificiais decorrentes de vazamento de informação, assegurando que as conclusões estejam fundamentadas em relações reais entre contexto e severidade dos acidentes.

In [28]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight="balanced"
)

rf.fit(log_reg_clf.named_steps["preprocessor"].transform(X_train), y_train)


,n_estimators,200
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [29]:
import pandas as pd
import numpy as np

feature_names = log_reg_clf.named_steps["preprocessor"].get_feature_names_out()
importances = rf.feature_importances_

rf_df = pd.DataFrame({
    "feature": feature_names,
    "importancia": importances
}).sort_values(by="importancia", ascending=False)

rf_df.head(15)


,feature,importancia
1,num__km,0.136898
0,num__br,0.094441
3,num__veiculos,0.036648
26,cat__sentido_via_Decrescente,0.019671
25,cat__sentido_via_Crescente,0.019545
18,cat__causa_acidente_N�o guardar dist�ncia de s...,0.016459
27,cat__condicao_metereologica_Ceu Claro,0.016442
40,cat__tracado_via_Reta,0.015249
32,cat__condicao_metereologica_Nublado,0.013470
16,cat__causa_acidente_Falta de aten��o,0.013456


In [30]:
from sklearn.metrics import classification_report, confusion_matrix

# Transformar X com o preprocessador da logística
X_test_proc = log_reg_clf.named_steps["preprocessor"].transform(X_test)

# Previsões
y_pred_rf = rf.predict(X_test_proc)

# Relatório completo
print("=== RANDOM FOREST ===")
print(classification_report(y_test, y_pred_rf))

# Matriz de confusão
print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred_rf))


=== RANDOM FOREST ===
              precision    recall  f1-score   support

           0       0.67      0.83      0.74      9744
           1       0.56      0.35      0.43      6219

    accuracy                           0.64     15963
   macro avg       0.61      0.59      0.59     15963
weighted avg       0.63      0.64      0.62     15963

Matriz de confusão:
[[8040 1704]
 [4029 2190]]


In [31]:
top_features = rf_df.head(10)["feature"].tolist()
from sklearn.feature_selection import SelectFromModel

selector = SelectFromModel(rf, threshold="median", prefit=True)
X_train_sel = selector.transform(log_reg_clf.named_steps["preprocessor"].transform(X_train))
X_test_sel  = selector.transform(log_reg_clf.named_steps["preprocessor"].transform(X_test))


In [32]:
from sklearn.linear_model import LogisticRegression

lr_sel = LogisticRegression(max_iter=1000, random_state=42)
lr_sel.fit(X_train_sel, y_train)

y_pred_red = lr_sel.predict(X_test_sel)

from sklearn.metrics import classification_report, confusion_matrix

print("=== REGRESSÃO LOGÍSTICA (MODELO REDUZIDO) ===")
print(classification_report(y_test, y_pred_red))
print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred_red))


=== REGRESSÃO LOGÍSTICA (MODELO REDUZIDO) ===
              precision    recall  f1-score   support

           0       0.65      0.87      0.74      9744
           1       0.57      0.27      0.36      6219

    accuracy                           0.63     15963
   macro avg       0.61      0.57      0.55     15963
weighted avg       0.62      0.63      0.60     15963

Matriz de confusão:
[[8463 1281]
 [4549 1670]]


In [33]:

rf_red = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight="balanced"
)
rf_red.fit(X_train_sel, y_train)
y_pred_rf_red = rf_red.predict(X_test_sel)

print("\n=== RANDOM FOREST (REDUZIDA) ===")
print(classification_report(y_test, y_pred_rf_red))
print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred_rf_red))




=== RANDOM FOREST (REDUZIDA) ===
              precision    recall  f1-score   support

           0       0.66      0.80      0.73      9744
           1       0.54      0.37      0.44      6219

    accuracy                           0.63     15963
   macro avg       0.60      0.58      0.58     15963
weighted avg       0.62      0.63      0.61     15963

Matriz de confusão:
[[7797 1947]
 [3947 2272]]


## 6. Avaliação detalhada

Aqui geramos métricas mais completas (relatório de classificação e matriz de confusão)
para entender melhor o comportamento do baseline.

In [ ]:
comparacao = pd.DataFrame({
    "Modelo": ["Regressão Logística", "Random Forest", "LR Reduzido", "RF Reduzido"],
    "F1": [
        f1_score(y_test, y_pred),
        f1_score(y_test, y_pred_rf),
        f1_score(y_test, y_pred_red),
        f1_score(y_test, y_pred_rf_red)
    ],
    "Acurácia": [
        accuracy_score(y_test, y_pred),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_red),
        accuracy_score(y_test, y_pred_rf_red)
    ],
    "Precisão": [
        precision_score(y_test, y_pred),
        precision_score(y_test, y_pred_rf),
        precision_score(y_test, y_pred_red),
        precision_score(y_test, y_pred_rf_red)
    ],
    "Recall": [
        recall_score(y_test, y_pred),
        recall_score(y_test, y_pred_rf),
        recall_score(y_test, y_pred_red),
        recall_score(y_test, y_pred_rf_red)
    ]
})




,Modelo,F1,Acurácia,Precisão,Recall
0,Regressão Logística,0.382192,0.640105,0.576948,0.285737
1,Random Forest,0.433106,0.640857,0.562404,0.352147
2,LR Reduzido,0.364231,0.634780,0.565910,0.268532
3,RF Reduzido,0.435332,0.630771,0.538516,0.365332


## Comparação entre Modelos e Seleção do Melhor Modelo até então

    Foram avaliados quatro cenários de modelagem: Regressão Logística e Random Forest utilizando o conjunto completo de variáveis, bem como versões reduzidas baseadas em seleção de atributos por importância. Os resultados indicaram desempenho superior dos modelos Random Forest em relação à Regressão Logística, evidenciando maior capacidade dos métodos de ensemble em capturar padrões não lineares associados à gravidade dos acidentes.
    A Random Forest apresentou F1-score significativamente superior, indicando melhor equilíbrio entre precisão e sensibilidade para a classe de interesse (acidentes graves). Observou-se também que a redução de variáveis impactou negativamente a regressão logística, mas não comprometeu o desempenho da Random Forest, cujo F1-score manteve-se praticamente inalterado após a remoção de variáveis menos importantes.
    Dessa forma, o Random Forest reduzido foi adotado em relação a regressão tanto reduzida quanto não, por oferecer a melhor relação entre desempenho preditivo e complexidade do modelo, utilizando um número menor de variáveis sem perda de qualidade na detecção de acidentes graves. Portanto iremos descartar a Regressão e testar outros modelos para fazer ensemble posterior

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, precision_score, recall_score

# MLP treinado nas features reduzidas
mlp_red = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation="relu",
    solver="adam",
    alpha=1e-4,
    batch_size=256,
    learning_rate="adaptive",
    max_iter=100,
    random_state=42,
    early_stopping=True,
    n_iter_no_change=5,
    validation_fraction=0.1
)

mlp_red.fit(X_train_sel, y_train)

y_pred_mlp_red = mlp_red.predict(X_test_sel)

print("=== MLP (REDUZIDO) ===")
print(classification_report(y_test, y_pred_mlp_red))
print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred_mlp_red))


=== MLP (REDUZIDO) ===
              precision    recall  f1-score   support

           0       0.67      0.85      0.75      9744
           1       0.59      0.34      0.43      6219

    accuracy                           0.65     15963
   macro avg       0.63      0.59      0.59     15963
weighted avg       0.64      0.65      0.62     15963

Matriz de confusão:
[[8266 1478]
 [4130 2089]]


In [ ]:
X_train_proc = log_reg_clf.named_steps["preprocessor"].transform(X_train)
X_test_proc  = log_reg_clf.named_steps["preprocessor"].transform(X_test)

# MLP treinado nas features completas
mlp_full = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation="relu",
    solver="adam",
    alpha=1e-4,
    batch_size=256,
    learning_rate="adaptive",
    max_iter=100,
    random_state=42,
    early_stopping=True,
    n_iter_no_change=5,
    validation_fraction=0.1
)

mlp_full.fit(X_train_proc, y_train)
y_pred_mlp_full = mlp_full.predict(X_test_proc)

print("=== MLP (COMPLETO) ===")
print(classification_report(y_test, y_pred_mlp_full))


=== MLP (COMPLETO) ===
              precision    recall  f1-score   support

           0       0.66      0.86      0.75      9744
           1       0.59      0.32      0.42      6219

    accuracy                           0.65     15963
   macro avg       0.63      0.59      0.58     15963
weighted avg       0.64      0.65      0.62     15963



In [41]:
mlp_full_row = {
    "Modelo": "MLP Completo",
    "F1": f1_score(y_test, y_pred_mlp_full),
    "Acurácia": accuracy_score(y_test, y_pred_mlp_full),
    "Precisão": precision_score(y_test, y_pred_mlp_full),
    "Recall": recall_score(y_test, y_pred_mlp_full)
}

mlp_red_row = {
    "Modelo": "MLP Reduzido",
    "F1": f1_score(y_test, y_pred_mlp_red),
    "Acurácia": accuracy_score(y_test, y_pred_mlp_red),
    "Precisão": precision_score(y_test, y_pred_mlp_red),
    "Recall": recall_score(y_test, y_pred_mlp_red)
}

comparacao_mlp = pd.concat(
    [comparacao, pd.DataFrame([mlp_full_row, mlp_red_row])],
    ignore_index=True
)

comparacao_mlp


,Modelo,F1,Acurácia,Precisão,Recall
0,Regressão Logística,0.382192,0.640105,0.576948,0.285737
1,Random Forest,0.433106,0.640857,0.562404,0.352147
2,LR Reduzido,0.364231,0.634780,0.565910,0.268532
3,RF Reduzido,0.435332,0.630771,0.538516,0.365332
4,MLP Completo,0.415441,0.649001,0.591503,0.320148
5,MLP Reduzido,0.426936,0.648688,0.585646,0.335906


## Comparação de Modelos e Escolha Final

    Foram avaliados seis modelos preditivos até então: Regressão Logística, Random Forest e MLP, cada qual treinado usando o conjunto completo de variáveis e uma versão reduzida baseada em seleção de atributos. Os resultados demonstraram clara superioridade dos modelos mais complexos em relação ao modelo baseline.
    A Regressão Logística apresentou desempenho inferior, especialmente no recall da classe grave, indicando limitação na captura de interações não lineares. A introdução da Random Forest elevou significativamente o F1-score e a sensibilidade para acidentes graves, confirmando a capacidade de modelos de ensemble em modelar padrões mais complexos.
    O MLP apresentou desempenho intermediário, superando a regressão logística mas sem alcançar o melhor desempenho da Random Forest, especialmente em termos de recall. Além disso, o MLP demanda maior custo computacional e apresenta menor interpretabilidade.
    A versão reduzida da Random Forest manteve desempenho equivalente ao modelo completo, com ligeira superioridade no F1-score e maior recall, demonstrando robustez frente à redução dimensional. Assim, o modelo Random Forest reduzido foi até então o melhor modelo do estudo por oferecer o melhor equilíbrio entre desempenho e simplicidade.

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
import pandas as pd

proba_rf  = rf_red.predict_proba(X_test_sel)[:, 1]
proba_mlp = mlp_red.predict_proba(X_test_sel)[:, 1]

proba_ensemble = (proba_rf + proba_mlp) / 2

threshold = 0.425
y_pred_ens_red = (proba_ensemble >= threshold).astype(int)

print("=== ENSEMBLE RF REDUZIDA + MLP REDUZIDA ===")
print(classification_report(y_test, y_pred_ens_red))
print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred_ens_red))


=== ENSEMBLE RF REDUZIDA + MLP REDUZIDA ===
              precision    recall  f1-score   support

           0       0.70      0.69      0.70      9744
           1       0.53      0.54      0.54      6219

    accuracy                           0.63     15963
   macro avg       0.62      0.62      0.62     15963
weighted avg       0.64      0.63      0.64     15963

Matriz de confusão:
[[6751 2993]
 [2835 3384]]


In [72]:
ensemble_row = {
    "Modelo": "Ensemble RF+MLP (Reduzido)",
    "F1": f1_score(y_test, y_pred_ens_red),
    "Acurácia": accuracy_score(y_test, y_pred_ens_red),
    "Precisão": precision_score(y_test, y_pred_ens_red),
    "Recall": recall_score(y_test, y_pred_ens_red)
}

comparacao_final = pd.concat(
    [comparacao_mlp, pd.DataFrame([ensemble_row])],
    ignore_index=True
)

comparacao_final


,Modelo,F1,Acurácia,Precisão,Recall
0,Regressão Logística,0.382192,0.640105,0.576948,0.285737
1,Random Forest,0.433106,0.640857,0.562404,0.352147
2,LR Reduzido,0.364231,0.634780,0.565910,0.268532
3,RF Reduzido,0.435332,0.630771,0.538516,0.365332
4,MLP Completo,0.415441,0.649001,0.591503,0.320148
5,MLP Reduzido,0.426936,0.648688,0.585646,0.335906
6,Ensemble RF+MLP (Reduzido),0.537313,0.634906,0.530657,0.544139


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# ===========================
# ENSEMBLE COM 3 MODELOS
# ===========================

# Probabilidade da classe GRAVE (classe = 1)
proba_lr  = lr_sel.predict_proba(X_test_sel)[:, 1]
proba_rf  = rf_red.predict_proba(X_test_sel)[:, 1]
proba_mlp = mlp_red.predict_proba(X_test_sel)[:, 1]

# Ensemble por média simples
proba_ens3 = 0.2 * proba_lr + 0.4 * proba_rf + 0.4 * proba_mlp


# Classe final
threshold = 0.42
y_pred_ens3 = (proba_ens3 >= threshold).astype(int)


print("=== ENSEMBLE (LOGÍSTICA + RF + MLP) - REDUZIDO ===")
print(classification_report(y_test, y_pred_ens3))
print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred_ens3))


ensemble3_row = {
    "Modelo": "Ensemble LR+RF+MLP (Reduzido)",
    "F1": f1_score(y_test, y_pred_ens3),
    "Acurácia": accuracy_score(y_test, y_pred_ens3),
    "Precisão": precision_score(y_test, y_pred_ens3),
    "Recall": recall_score(y_test, y_pred_ens3)
}

comparacao_final = pd.concat(
    [comparacao_final, pd.DataFrame([ensemble3_row])],
    ignore_index=True
)

comparacao_final


=== ENSEMBLE (LOGÍSTICA + RF + MLP) - REDUZIDO ===
              precision    recall  f1-score   support

           0       0.71      0.68      0.70      9744
           1       0.53      0.56      0.54      6219

    accuracy                           0.63     15963
   macro avg       0.62      0.62      0.62     15963
weighted avg       0.64      0.63      0.64     15963

Matriz de confusão:
[[6646 3098]
 [2732 3487]]


,Modelo,F1,Acurácia,Precisão,Recall
0,Regressão Logística,0.382192,0.640105,0.576948,0.285737
1,Random Forest,0.433106,0.640857,0.562404,0.352147
2,LR Reduzido,0.364231,0.634780,0.565910,0.268532
3,RF Reduzido,0.435332,0.630771,0.538516,0.365332
4,MLP Completo,0.415441,0.649001,0.591503,0.320148
5,MLP Reduzido,0.426936,0.648688,0.585646,0.335906
6,Ensemble RF+MLP (Reduzido),0.537313,0.634906,0.530657,0.544139
7,Ensemble LR+RF+MLP (Reduzido),0.544674,0.634780,0.529537,0.560701


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# ===========================
# ENSEMBLE COM 3 MODELOS TUNING DE PARAMETROS
# ===========================

# Probabilidades individuais
proba_lr  = lr_sel.predict_proba(X_test_sel)[:, 1]
proba_rf  = rf_red.predict_proba(X_test_sel)[:, 1]
proba_mlp = mlp_red.predict_proba(X_test_sel)[:, 1]

# Grade de testes
weights_lr = [0.1, 0.2, 0.3]
weights_rf = [0.3, 0.4, 0.5]
weights_mlp = [0.2, 0.3, 0.4]
thresholds = np.arange(0.30, 0.61, 0.02)

results = []

for w_lr in weights_lr:
    for w_rf in weights_rf:
        for w_mlp in weights_mlp:
            if abs((w_lr + w_rf + w_mlp) - 1.0) > 1e-6:
                continue

            for th in thresholds:
                proba_ens = w_lr * proba_lr + w_rf * proba_rf + w_mlp * proba_mlp
                y_pred = (proba_ens >= th).astype(int)

                f1 = f1_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                acc = accuracy_score(y_test, y_pred)

                results.append([w_lr, w_rf, w_mlp, th, f1, precision, recall, acc])

# DataFrame com todos os combos
df_results = pd.DataFrame(results, columns=[
    "w_LR", "w_RF", "w_MLP", "threshold",
    "F1", "Precisão", "Recall", "Acurácia"
])

# Ordenar pelo F1
df_results_sorted = df_results.sort_values(by="F1", ascending=False)

# Mostrar top 10
df_results_sorted.head(10)


,w_LR,w_RF,w_MLP,threshold,F1,Precisão,Recall,Acurácia
49,0.3,0.3,0.4,0.32,0.595025,0.467789,0.817334,0.566560
16,0.2,0.4,0.4,0.30,0.594610,0.458362,0.846117,0.550523
64,0.3,0.4,0.3,0.30,0.594525,0.457557,0.848529,0.549082
48,0.3,0.3,0.4,0.30,0.594462,0.455817,0.854317,0.545887
80,0.3,0.5,0.2,0.30,0.592614,0.458605,0.837273,0.551525
65,0.3,0.4,0.3,0.32,0.592457,0.468449,0.805757,0.568126
32,0.2,0.5,0.3,0.30,0.592068,0.458871,0.834218,0.552152
81,0.3,0.5,0.2,0.32,0.591351,0.470036,0.797073,0.570820
0,0.1,0.5,0.4,0.30,0.591185,0.458981,0.830359,0.552590
33,0.2,0.5,0.3,0.32,0.591132,0.470376,0.795305,0.571384


## Ajuste de Pesos e Otimização do Ensemble

    Após a construção do ensemble entre Regressão Logística, Random Forest e MLP reduzidos, foi realizado um processo de ajuste empírico de pesos e do limiar de decisão, visando maximizar o F1-score da classe grave. Foram testadas diversas combinações de pesos e thresholds, selecionando-se a configuração que apresentou melhor desempenho global.
    A melhor configuração obtida atribuiu pesos de 0.3 para a Regressão Logística, 0.3 para a Random Forest e 0.4 para a MLP, utilizando limiar de classificação igual a 0.32. Essa configuração alcançou F1-score de 0.595 e recall superior a 0.81 para a classe de acidentes graves.
    O resultado indica que o ensemble se tornou significativamente mais sensível na detecção de eventos críticos, ainda que à custa de redução da precisão. No contexto do problema, esse trade-off é aceitável e desejável, uma vez que o custo de não identificar um acidente grave é maior que o custo de um falso positivo.

## Modelo Final Recomendado

    O ensemble otimizado entre Regressão Logística, Random Forest e MLP foi selecionado como modelo final do estudo, por apresentar o melhor equilíbrio entre desempenho e sensibilidade à classe de interesse. Sua alta capacidade de detecção de acidentes graves demonstra grande potencial para uso em sistemas de apoio à decisão e monitoramento de segurança.

# CONCLUSÃO DAS HIPÓTESES E RESPOSTAS ÀS PERGUNTAS DE PESQUISA
## Respostas às Perguntas de Pesquisa
### Pergunta 1
Quais fatores estão mais associados à ocorrência de acidentes graves nas rodovias federais em 2007?

    A análise dos modelos reduzidos e da importância das variáveis na Random Forest identificou que a gravidade dos acidentes não é aleatória, estando fortemente associada a fatores estruturais, ambientais e comportamentais. Os principais fatores identificados foram:
    Causa do acidente, destacando-se ingestão de álcool, ultrapassagens indevidas, falta de atenção e desobediência à sinalização;
    Tipo de pista, sendo vias simples consideravelmente mais propensas a acidentes graves;
    Número de veículos envolvidos, indicando maior severidade em colisões múltiplas;
    Condições meteorológicas, especialmente chuva e tempo nublado;
    Uso do solo, com maior risco em áreas rurais;
    Faixa horária e fase do dia, evidenciando maior gravidade durante horários noturnos e de madrugada;
    Traçado da via, sobretudo em vias retas de alta velocidade.
    Esses fatores refletem tanto comportamento humano de risco quanto características da infraestrutura e do ambiente físico das rodovias, demonstrando que acidentes graves decorrem da combinação de múltiplos fatores contextuais.

### Pergunta 2
É possível prever a gravidade de um acidente com base nas características da ocorrência?

    Sim. Os resultados demonstram que é plenamente possível prever a gravidade dos acidentes a partir das características da ocorrência. O ensemble otimizado entre Regressão Logística, Random Forest e MLP apresentou desempenho significativamente superior aos modelos individuais, alcançando:
    F1-score de 0.595
    Recall de 0.817
    Precisão de 0.468
    O elevado recall indica que o modelo é altamente eficaz na identificação de acidentes graves, tornando-se apropriado para aplicações em cenários de alto risco, como planejamento de fiscalização, triagem de ocorrências e prevenção viária.
    A performance geral confirma que técnicas de aprendizado de máquina conseguem capturar padrões relevantes nos dados, mesmo diante de desbalanceamento de classes.


## Validação das Hipóteses

### Hipótese 1 (H1)
Fatores ambientais e temporais influenciam a gravidade dos acidentes?

    Sim, as variáveis relacionadas a clima, período do dia, uso do solo e localização figuraram entre as mais relevantes nos modelos reduzidos, confirmando sua influência direta sobre a severidade dos acidentes.

### Hipótese 2 (H2)
O número de veículos e o tipo de pista são preditores significativos da gravidade?

    Sim, tanto o número de veículos quanto o tipo de pista foram mantidos na seleção de variáveis e apresentaram influência consistente nos modelos, comprovando que infraestrutura e volume de tráfego são fatores críticos na severidade dos sinistros.

### Hipótese 3 (H3)
Modelos de ensemble apresentam desempenho superior na previsão da gravidade.

    Sim, o ensemble otimizado superou todos os modelos individuais em F1-score e recall, demonstrando que a combinação de modelos com vieses distintos aumenta robustez e capacidade de generalização.

## Considerações Finais

    Este trabalho demonstrou que a gravidade de acidentes rodoviários pode ser predita de forma eficaz por meio de técnicas de Machine Learning, desde que haja seleção criteriosa de variáveis e abordagem adequada à modelagem.
    A introdução de modelos mais complexos elevou substancialmente o desempenho preditivo em relação ao baseline, e a utilização de técnicas de ensemble mostrou-se decisiva para atingir o melhor resultado. O ganho mais expressivo ocorreu na capacidade de detectar corretamente acidentes graves, que é o principal objetivo prático em contextos de segurança.
    O modelo final selecionado foi o ensemble entre Regressão Logística, Random Forest e MLP otimizado, utilizando pesos calculados empiricamente e limiar de decisão ajustado, resultando no melhor equilíbrio entre desempenho estatístico e utilidade prática.
    Conclui-se que o uso de aprendizado de máquina é viável, eficaz e altamente recomendável como ferramenta de suporte à decisão em políticas públicas de segurança viária, podendo apoiar desde a alocação de recursos até a definição de estratégias preventivas em rodovias federais.